In [ ]:
import requests
import json

We don't want you all to have access to the password, so it is stored locally on my machine.

We just read it from that file before using it:

In [ ]:
with open('reddit-pass', 'r') as file:
    psswd = file.read().replace('\n','')

All of the information about how to access Reddit's API can be found on their documentation page [https://www.reddit.com/dev/api/](https://www.reddit.com/dev/api/).

Unfortunately there are no universal rules for all APIs. So you'll have to get used to reading the documentation for the service/API that you're trying to use.

Below we have to provide the two hashes (both are provided by Reddit when we create an 'App'):

* Our ID Hash
* Our secret Hash

In [ ]:
auth = requests.auth.HTTPBasicAuth("rGQNNzCHhqnDDav5zERGEA", "qNV2hJnaL7CFM3xEf7qNitFkn1En5g")

In addition to the 'App' identity and secret, Reddit also requires that we identify which user has created this application.

In [ ]:
data = {'grant_type': 'password',
        'username':    "cmsc320-demo",
        'password':    psswd }

Lastly, we need to name our bot/application. I've decided to name it 'cmsc320/1.0', because I'm boring.

In [ ]:
init_headers = {'User-Agent': 'cmsc320/1.0'}

Putting it all together, we create a POST request. This sends all of our data (`auth`, `data`, and `headers`) to Reddit.

The Reddit servers can then determine whether we should be allowed to access information about the `cmsc320-demo` account. If so, we get a response that includes an _access token_.

Notice that we throw an exception if the status code of our response does not signify success (200).

In [ ]:
r = requests.post('https://www.reddit.com/api/v1/access_token',
                  auth=auth, data=data, headers=headers)

if r.status_code != 200:
    raise requests.ConnectionError("We can't continue if the request was not successful. :(")

We don't have to care about the value of the access TOKEN, it's just a way that we can show the Reddit servers "hey, we already authenticated".

Different APIs will have tokens expire at different rates.

It's not clear how long a TOKEN from Reddit lasts, because it's not in the documentation! Could be 30 minutes, could be a few hours, could be a few years! lol

If your code seems to be working, and then suddenly you start getting 401 or 403 responses from the server, you may need to re-authenticate.

In [ ]:
REDDIT_TOKEN = r.json()['access_token']

REDDIT_TOKEN

The syntax below is useful to know, so let's have a go:

When you have a dictionary (`init_headers`, in this case), and you would like _each item in the dictionary_ to become an item _in a new dictionary_, you can use the `**` syntax. Which roughly translates to "take each item in the following dictionary, and insert it into the dictionary I'm currently creating".

In [ ]:
headers = {**init_headers, **{'Authorization':f"bearer {REDDIT_TOKEN}"}}

Just to show you the result, we print it out below.
We have a single level dictionary that is made up of each element from the two dictionaries we used `**` on.

In this case it's not very interesting, but we wanted to show you this syntax.

One question to ask yourself: What would happen if we just removed both of the `**`s above?

In [ ]:
headers

Now we can start _using_ the API.

For example, the reddit API allows us to ask questions about the user we authenticated as. Below we as for information about `cmsc320-demo` as a user, and we ask for information regarding their 'karma' (the points system that reddit uses to encourage engagement).

Documentation for these API calls/endpoints are as follows:

* [endpoint for information on 'me'](https://www.reddit.com/dev/api#GET_api_v1_me)

* [endpoint for information on 'my karma'](https://www.reddit.com/dev/api#GET_api_v1_me_karma)

In [ ]:
me = requests.get('https://oauth.reddit.com/api/v1/me', headers=headers)
karma = requests.get('https://oauth.reddit.com/api/v1/me/karma', headers=headers)

In [ ]:
print(karma.json())

In [ ]:
len(me.json())

We can also get information about specific subreddits (specialized communities for a specific topic).

There is a subreddit where users can submit short stories. We can ask for the 'top' submissions.

The 'top' submissions will be the current popular submissions on the subreddit. The documentation does not specify how far back in time it will consider for a 'top' submission, why would it? That would be useful. In fact, the documentation barely says anything [documentation for 'top'](https://www.reddit.com/dev/api#GET_top)

In [ ]:
shorties      = requests.get("https://oauth.reddit.com/r/shortstories/top.json", headers=headers)
shorties_json = shorties.json()
len(str(shorties_json))

As you can see, there is a lot of information that is provided. While you can print it out and investigate, it's better to use the documentation to see how you might use it.

Another thing we can ask of a subreddit is "what are the top submissions of all time":

In [ ]:
shortiesGOAT = requests.get("https://oauth.reddit.com/r/shortstories/top.json?limit=25&t=all", headers=headers)
goat_json    = shortiesGOAT.json()
len(str(goat_json))

Notice that's bigger!

If we look at the documentation for a `listing` (the thing that reddit gives up back), we can see how to look things up in that listing.

[documentation for a 'listing'](https://www.reddit.com/dev/api#listings).

However the documentation is pretty sparse. Another approach is to list the keys in the resulting dictionary to see what might be available/interesting:

In [ ]:
shorties_json.keys()

`data` seems interesting, let's see what's in there:

In [ ]:
shorties_json['data'].keys()

so on and so forth...

It can be a tedious process, made less tedious if the documentation is thorough. Think about this when you're wondering whether it's worth documenting your own code :)

In [ ]:
all_posts = shorties.json()['data']['children']

for post in all_posts:
    print("# " + post['data']['title'])

In [ ]:
all_posts = shortiesGOAT.json()['data']['children']

for post in all_posts:
    print("# " + post['data']['title'])